In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdrivermanager.chrome import ChromeDriverManager

ModuleNotFoundError: No module named 'webdrivermanager'

In [3]:
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests
import time
import csv

In [ ]:
# arielle
start_page = "https://www.fdle.state.fl.us/MCICSearch/Search.asp"

driver = webdriver.Chrome()
driver.get(start_page)

# use Select () to get the select drop-down menu
category_menu = Select(driver.find_element(By.ID, "category"))

# go to 5th option in the select element
category_menu.select_by_index(4)


#now press search button, select by XPATH
search_button = driver.find_element(By.XPATH, "//*[@id='SecondaryPageRightColumnInnerContent']/div[1]/div[19]/img")
search_button.click()

#list to save all the missing person's data
data_list = []

#this function extracts the missing person data from the table using beautifulsoup
def extract_data(driver, data_list):
    # extract the HTML source code
    html = driver.page_source

    # parse the HTML using Beautiful Soup
    soup = BeautifulSoup(html, "html.parser")

    # find the table element by its id or xpath
    table = soup.find("table", class_="DataTable")
    # loop through the rows and extract the data
    for row in table.find_all("tr"):
        cells = row.find_all("td")
        #list to save each missing person data as a list into the main data_list
        missing_person = []
        for index, cell in enumerate(cells):
            # extract data only from the 2nd, 3rd, and last cells from the missing data table
            if index == 1 or index == 2 or index == len(cells) - 1:
                # get the text of the cell
                data = cell.text
                #strip the data and append into the list
                missing_person.append(data.strip())
        data_list.append(missing_person)


# this function uses a for loop to jump to the next page and extract data for the particular page
def next_page(driver):
    for i in range(3, 31):
        # using Select () to get the select drop-down menu for next page
        next_page = Select(driver.find_element(By.ID, "PageListTop"))
        next_page.select_by_index(i)
        
        #extract data will append the current page data to data_list
        extract_data(driver, data_list)
        time.sleep(1)
        
    #these 3 lines gets the last page's missing persons data
    next_page = driver.find_element(By.XPATH, "//*[@id='SecondaryPageRightColumnInnerContent']/div[2]/strong[5]/a")
    next_page.click()
    extract_data(driver, data_list)
        

def create_missing_person_csv(data):
    # Remove the first 3 elements from the data list
    columns = ['Missing Since', 'Name', 'Missing From']
    data = data[1:]
#     print(data)

    # Open a new file for writing
    with open('missing_person_data.csv', mode='w', newline='') as file:
        # Create a CSV writer object
        writer = csv.writer(file)

        # Write the header row
        writer.writerow(["Name", "Missing Since", "Missing From"])

        # Loop through the data list and write each row to the CSV file
        for missing_person in data:
            if len(missing_person) == 3:
                print(missing_person)
                if missing_person != columns:
#                     print(missing_person)
                    name = missing_person[1]
#                     print("Name: ", name)
                    missing_since = missing_person[0]
#                     print("Missing Since: ", missing_since)
                    missing_from = missing_person[2]
#                     print("Missing From: ", missing_from)

                    writer.writerow([name, missing_since, missing_from])
            else:
                print("Data incomplete")

    print("CSV file created successfully!")

next_page(driver)
create_missing_person_csv(data_list)

